# Production Technology - Martin RE

The dataset contains `N = 441` firms observed over `T = 12` years, 1968-1979. There variables are: 
* `lcap`: Log of capital stock, $k_{it}$ 
* `lemp`: log of employment, $\ell_{it}$ 
* `ldsa`: log of deflated sales, $y_{it}$
* `year`: the calendar year of the observation, `year` $ = 1968, ..., 1979$, 
* `firmid`: anonymized indicator variable for the firm, $i = 1, ..., N$, with $N=441$. 

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from numpy import linalg as la
import pandas as pd
from scipy import stats
from scipy.stats import chi2
from io import StringIO
from tabulate import tabulate
import LinearModelsWeek3 as lm
from matplotlib import pyplot as plt
import seaborn as sns

#Supress Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



In [2]:
dat = pd.read_csv('firms.csv')

# Converting data to numpy format 

In [3]:
#Beholder kun 3 år

dat = dat[(dat['year'] >= 1968) & (dat['year'] <= 1970)]
dat=dat.reset_index(drop=True)
dat

,firmid,year,lcap,lemp,ldsa
0,1,1968,0.998602,-0.242185,0.349053
1,1,1969,0.925214,-0.241278,0.312492
2,1,1970,0.879616,-0.265134,0.347566
3,2,1968,-0.069588,-0.323021,-0.945831
4,2,1969,-0.056724,-0.358177,-1.143830
...,...,...,...,...,...
1318,440,1969,-0.228757,0.031242,-0.246864
1319,440,1970,-0.038354,0.062158,-0.345710
1320,441,1968,-1.618390,-1.944210,-2.032340
1321,441,1969,-1.635030,-1.856580,-2.011210


In [4]:
dat.ldsa.values.shape

(1323,)

In [5]:
N = dat.firmid.unique().size
T = dat.year.unique().size
assert dat.shape[0] == N*T, f'Error: data is not a balanced panel'
print(f'Data has N={N} and T={T}')

Data has N=441 and T=3


Extract data from `pandas` to `numpy` arrays. 

In [6]:
y = dat.ldsa.values.reshape((N*T,1))

#Should we not stack the ones on there as well for the constant? - What do we do with the constant in RE? 


ones = np.ones((N*T,1))
l = dat.lemp.values.reshape((N*T,1))
k = dat.lcap.values.reshape((N*T,1))
x = np.hstack([l, k])

x.shape

(1323, 2)

# FE AND FD RESULTS

In [7]:
# Create transformation matrix
def demeaning_matrix(T):
    ones_T = np.ones(T)
    Q_T = np.identity(T) - (1 / T) * np.outer(ones_T, ones_T)
    return Q_T

# Print the matrix
Q_T = demeaning_matrix(T)
print(f'Demeaning matrix for T={T} \n', Q_T)

# Transform the data
y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, x)

#print x_demean
print(x_demean.shape)

label_y = 'log Deflated sales'
label_x = [
    'log Employment', 
    'log Adjusted Capital Stock',  
]
# Create function to check rank of demeaned matrix, and return its eigenvalues.
def check_rank(x):
    print(f'Rank of demeaned x: {la.matrix_rank(x)}')
    lambdas, V = la.eig(x.T@x)
    np.set_printoptions(suppress=True)  # This is just to print nicely.
    print(f'Eigenvalues of within-transformed x: {lambdas.round(decimals=0)}')

# Check rank of demeaned x
check_rank(x_demean)

# Estimate using the demeaned variables, y_demean and x_demean
fe_result = lm.estimate(y_demean, x_demean, transform='fe', T=T)

# Print results
lm.print_table((label_y, label_x), fe_result, title='FE regression', floatfmt='.4f')
b_hat=fe_result['b_hat']

Demeaning matrix for T=3 
 [[ 0.66666667 -0.33333333 -0.33333333]
 [-0.33333333  0.66666667 -0.33333333]
 [-0.33333333 -0.33333333  0.66666667]]
(1323, 2)
Rank of demeaned x: 2
Eigenvalues of within-transformed x: [9. 5.]
FE regression
Dependent variable: log Deflated sales

                              Beta      Se    t-values
--------------------------  ------  ------  ----------
log Employment              0.6004  0.0346     17.3515
log Adjusted Capital Stock  0.0502  0.0382      1.3143
R² = 0.284
σ² = 0.008


# Random effects estimation
Last time we briefly discussed how the presence of unobserved heterogeneity can cause estimators to be biased. Consider the following model,

For RE, it must hold that: 
$E[c_i\boldsymbol{x}_{it}]=0$ for all $t$, so $\boldsymbol{\beta}$ can be consistently estimated by pooled OLS (POLS) and random effects (RE) if the respective rank conditions hold. 


There are some (time-invariant) variables that we could control for, for example if we believe afro americans are more or less prone to unionizing, because of some social economic factors. We can look at the conditional mean for Blacks and Hispanics.

# Part 2: The random effects (RE) estimator.
In part 1 we used two methods to remove unobserved heterogeneity from each person. Now, what if $E[union_{it}c_i] = 0$? Then POLS is consistent, but not efficient, since POLS is not using the panel structure of the data. We can therefore do better with the RE estimator.

## A short introduction to the RE estimator
With the FE and FD estimators, we estimate them by OLS, but by first transforming them in a specific way. We can do the same for RE, but our mission is no longer to transform away the fixed effects, but rather to estimate the following model,

$$\check{y}_{it} = \mathbf{\check{x}}_{it}\boldsymbol{\beta} + \check{v}_{it},\tag{6}$$ 

 $\check{y}_{it} = y_{it} - \hat{\lambda}\bar{y}_{it}$, $\mathbf{\check{x}}_{it} = \mathbf{x}_{it} - \hat{\lambda}\mathbf{\bar{x}}_{it}$, and $\check{v}_{it} = v_{it} - \hat{\lambda}\bar{v}_{it}$, where we have gathered the errors $v_{it} = c_i + u_{it}$. We are *"quasi-demeaning"* the variables, by premultiplying the means by $\hat{\lambda}$ (see Wooldridge p. 326-328).

 Our challenge is thus to estimate this $\lambda$, which we can construct in the following way:

$$\hat{\lambda} = 1 - \sqrt{\frac{\widehat{\sigma}_{u}^{2}}{(\widehat{\sigma}_{u}^{2} + T\widehat{\sigma}_{c}^{2})}}, $$

where $\widehat{\sigma}_{u}^{2}$ can be estimated from the fixed effects regression, and $\hat{\sigma}_{c}^{2}$ can be constructed as  $\hat{\sigma}_{c}^{2} = \hat{\sigma}_{w}^{2} - \frac{1}{T}\hat{\sigma}_{u}^{2}$. Here $\hat{\sigma}_{w}^{2}$ is the error variance from the between estimator, 


$$
\hat{\sigma}_{w}^{2} = \frac{1}{N-K}\left(\bar{\mathbf{y}} - \mathbf{\bar{X}}\hat{\mathbf{\beta}}_{BE}\right)^{\prime}\left(\bar{\mathbf{y}} - \mathbf{\bar{X}}\hat{\mathbf{\beta}}_{BE}\right),
$$

where $\boldsymbol{\beta}_{BE}$ are the between estimater coefficients. The between-groups estimator is not something we have introduced before, but is attained by regressing the time-averaged outcomes $\overline{y}_i$ on the time-averaged regressors $\overline{\mathbf{x}}_i,i=1,2,\dotsc,N$.

*Note:* There are other procedures for estimating the variances. See Wooldridge p. 294-296 for more details. 

### Question 1: The Between Estimator
Estimate the between groups model, which is simply the average within each each individual,

$$
\bar{y}_{i} = \boldsymbol{\bar{x}}_{i}\boldsymbol{\beta} + c_i + \bar{u}_{i}.
$$

So instead of demeaning, like we did in FE, we just calculate the mean with the following transformation *vector* $\mathbf{P}_T$,

\begin{equation} 
\mathbf{P}_T \equiv \left( \frac{1}{T}, \frac{1}{T}, ..., \frac{1}{T} \right)_{1 \times T}  \notag
\end{equation}

In order to estimate eq. (3) with the between estimator. You need to perform the following steps:
* Create the mean vector `P`.
* mean `x` and `y` using the `perm` function and `P`.
* Regress `y_mean` on `x_mean`. Note that there are $N$ rows in each, not $NT$. 
* Print it out in a nice table.

In [15]:
# Transform the data
P_T = np.ones(T) /T 
P_T = P_T.reshape(1,-1)
y_mean = lm.perm(P_T, y)
x_mean = lm.perm(P_T, x)

# Estimate 
be_result = lm.estimate(y_mean, x_mean, transform='be', T=T)
lm.print_table((label_y, label_x), be_result, title="Between Estimator", floatfmt='.4f')

Between Estimator
Dependent variable: log Deflated sales

                              Beta      Se    t-values
--------------------------  ------  ------  ----------
log Employment              0.6856  0.0333     20.5565
log Adjusted Capital Stock  0.2778  0.0295      9.4042
R² = 0.921
σ² = 0.122


### Question 2
You should now have all the error variances that you need to calculate

$$\hat{\lambda} = 1 - \sqrt{\frac{\widehat{\sigma}_{u}^{2}}{(\widehat{\sigma}_{u}^{2} + T\widehat{\sigma}_{c}^{2})}}. $$

In [16]:
# Calculate lambda (note lambda is a reserved keyword in Python, so we use _lambda instead)
sigma2_u = fe_result['sigma2']
sigma2_w = be_result['sigma2']
sigma2_c = sigma2_w-(1/T)*sigma2_u 
_lambda = 1-np.sqrt(sigma2_u/(sigma2_u + T*sigma2_c))

# Print lambda 
print(f'Lambda is approximately equal to {_lambda.item():.4f}.')

Lambda is approximately equal to 0.8509.


### Question 3
Now we are finally ready to estimate eq. (3) with random effects. Since we have to use $\hat{\lambda}$ to quasi-demean within each individual, we again use the `perm` function. This time, we pass it the following transformation matrix,

$$
\mathbf{C}_{T}:=\mathbf{I}_{T} - \hat{\lambda}\mathbf{P}_{T},
$$

where $\mathbf{P}_{T}$ is the $1 \times T$ transformation vector we used earlier to calculate the mean of each person.

In [18]:
# Transform the data
C_T = np.identity(T) - _lambda*P_T
y_re = lm.perm(C_T, y)
x_re = lm.perm(C_T, x)

# Estimate 
re_result = lm.estimate(y_re, x_re, transform='re', T=T)
lm.print_table((label_y, label_x), re_result, title="Random Effects", floatfmt='.4f')
print("Lambda=", _lambda)

Random Effects
Dependent variable: log Deflated sales

                              Beta      Se    t-values
--------------------------  ------  ------  ----------
log Employment              0.6912  0.0235     29.4669
log Adjusted Capital Stock  0.2477  0.0214     11.6004
R² = 0.797
σ² = 0.008
Lambda= [[0.8509242]]


## Short introduction to Hausman test

It is evident from the previous question that RE has the advantage over FE that time-invariant variables are not demeaned away. But if $E[c_{i}\boldsymbol{x}_{it}] \neq \boldsymbol{0}$, then the RE estimator is inconsistent, where the FE estimator is consistent (but inefficient), assuming strict exogeneity.

We can use the results from the FE and RE estimations to test whether RE is consistent, by calculating the following test statistics,

$$
H := (\hat{\boldsymbol{\beta}}_{FE} - \hat{\boldsymbol{\beta}}_{RE})'[\widehat{\mathrm{avar}}(\hat{\boldsymbol{\beta}}_{FE}) - \widehat{\mathrm{avar}}(\hat{\mathbf{\beta}}_{RE})]^{-1}(\hat{\boldsymbol{\beta}}_{FE}-\hat{\boldsymbol{\beta}}_{RE})\overset{d}{\to}\chi_{M}^{2}, \tag{7}
$$
where M is the number of time-variant variables included in the test.

*Note 1*: The vector $\hat{\boldsymbol{\beta}}_{RE}$ excludes time invariant variables as these are not present in $\hat{\boldsymbol{\beta}}_{FE}$. <br>
*Note 2:* $\widehat{\mathrm{avar}}(\hat{\boldsymbol{\beta}}_{RE})$ means the RE covariance (but again, we only keep the rows and columns for time-variant variables)

#### Question 4: Comparing FE and RE
Use the results from the FE and RE esimtations to compute the Hausman test statistics in eq. (7).

* Start by calculating the differences in the FE and RE coefficients $\hat{\boldsymbol{\beta}}_{FE} - \hat{\boldsymbol{\beta}}_{RE}$ (remember to remove the time invariant variables from RE)
* Then calculate the differences in the covariances $\widehat{\mathrm{avar}}(\hat{\boldsymbol{\beta}}_{FE}) - \widehat{\mathrm{avar}}(\hat{\boldsymbol{\beta}}_{RE})$ (again, remember to remove the time invariant variables for RE estimates)
* You now have all the components to compute the Hausman test statistics in eq. (7)

In [19]:
# Unpack
b_fe = fe_result['b_hat'] # Fill in
b_re = re_result['b_hat']
cov_fe = fe_result['cov']
cov_re = re_result['cov'] # Do we have any time invariant variables? Don't think so

# Calculate the test statistic
b_diff = b_fe - b_re
cov_diff = cov_fe - cov_re
H = b_diff.T@la.inv(cov_diff)@b_diff

# Find critical value and p-value at 5% significance level of chi^2 with M degrees of freedom
M = 2
crit_val = stats.chi2.ppf(1 - 0.05, df=M)
p_val = 1 - stats.chi2.cdf(H.item() , df=M)

# Print the results
print(f'The test statistic is {H.item():.2f}.')
print(f'The critical value at a 5% significance level is {crit_val:.2f}.')
print(f'The p-value is {p_val:.8f}.')

The test statistic is 50.88.
The critical value at a 5% significance level is 5.99.
The p-value is 0.00000000.


Which assumption is tested by the Hausman test? What is the null hypothesis? Does the Hausman test you have conducted rely on any other assumptions (See Wooldridge, p. 328-331)? Based on your test result, which estimator would you use to estimate eq. (3)? Why?